<a href="https://colab.research.google.com/github/kStasya/Distributed_computing/blob/main/CUDA/CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Apr  6 15:19:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
import numpy as np
import time
from numba import jit

##Задание №2




In [ ]:
# without GPU
def task2_cpu(x):
  N = x.shape[0]
  b = np.zeros(N)
  for j in range(0, N):
    if (j==0):
      b[j] = (a[j] + a[j+1]) / 2
    elif (j==N-1):
      b[j] = (a[j] + a[j-1]) / 2
    else:
      b[j] = (a[j-1] + a[j] + a[j+1]) / 3
  return b

In [ ]:
# with GPU
@jit
def task2_gpu(x):
  N = x.shape[0]
  b = np.zeros(N)
  for j in range(0, N):
    if (j==0):
      b[j] = (a[j] + a[j+1]) / 2
    elif (j==N-1):
      b[j] = (a[j] + a[j-1]) / 2
    else:
      b[j] = (a[j-1] + a[j] + a[j+1]) / 3
  return b

In [ ]:
N = 16000
a = np.array([i for i in range(N)])

# without GPU
start = time.time()
b_1 = task2_cpu(a)
print("CPU. Elapsed time (with compilation): ", (time.time() - start), "s.", sep='')

# with GPU
start = time.time()
b_2 = task2_gpu(a)
print("GPU. Elapsed time (with compilation): ", (time.time() - start), "s.", sep='')

CPU. Elapsed time (with compilation): 0.013727903366088867s.
GPU. Elapsed time (with compilation): 0.6968071460723877s.


## Задание №4

In [ ]:
# without GPU
def task4_cpu(A, B):
  a, b, c = A.shape[0], A.shape[1], B.shape[1]
  C = np.zeros((a, c))
  for i in range(a):
    for j in range(b):
      for k in range(c):
        C[i][k] += A[i][j] * B[j][k]
  return C

In [ ]:
# with GPU
@jit
def task4_gpu(A, B):
  a, b, c = A.shape[0], A.shape[1], B.shape[1]
  C = np.zeros((a, c))
  for i in range(a):
    for j in range(b):
      for k in range(c):
        C[i][k] += A[i][j] * B[j][k]
  return C

In [ ]:
a, b, c = 10, 15, 16  # размерности матриц A[a x b], B[b x c]
A = np.random.rand(a, b) * 10
B = np.random.rand(b, c) * 10

start = time.time()
C = task4_cpu(A, B)
print("CPU. Elapsed time (with compilation): ", (time.time() - start), "s.", sep='')

start = time.time()
C = task4_gpu(A, B)
print("GPU. Elapsed time (with compilation): ", (time.time() - start), "s.", sep='')

CPU. Elapsed time (with compilation): 0.0036454200744628906s.
GPU. Elapsed time (with compilation): 0.27398157119750977s.


## Задание №5

In [ ]:
@jit
def task5(points):
  N = points.shape[0]
  return [np.sum(points.T[0])/N, np.sum(points.T[1])/N, np.sum(points.T[2])/N]

In [ ]:
@jit
def task5_axis(x):
  N = x.shape[0]
  return np.sum(x)/N

In [ ]:
N = 1000
points = np.array([np.random.rand(3) for i in range(N)])

start = time.time()
mean = task5(points)
print("One function. Elapsed time (with compilation): ", (time.time() - start), "s.", sep='')

start = time.time()
mean_x = task5_axis(points.T[0])
mean_y = task5_axis(points.T[1])
mean_z = task5_axis(points.T[2])
print("Functional decomposition. Elapsed time (with compilation): ", (time.time() - start), "s.", sep='')

One function. Elapsed time (with compilation): 0.22918367385864258s.
Functional decomposition. Elapsed time (with compilation): 0.0646357536315918s.


In [ ]:
def f(x):
  return x**3 - 3

# Основное задание
## Задание №2

Вычисляемый интеграл:
$$ \int_a^b (x^3 - 3)dx, $$
где $a=1, $ $b=3$.

Формула вычисления определенного интеграла методом трапеции:
$$ \int_{x_0}^{x_n}f(x)dx = \big(f(x_0) + f(x_n) + 2\sum_{i=1}^{n-1}f(x_i) \big) \cdot \frac{h}{2}, $$
где $ x_0 = a,$ $x_n = b,$ $x_i = a + \frac{i \cdot (b-a)}{n},$ $i = \overline{1, n-1}.$

In [ ]:
@jit
def task2(a, b, N):
  integral = (a**3 + b**3 - 6) / 2
  for i in range(1, N-1):
    integral += ((a + i * (b - a))**3 - 3) / N
  integral *= (b - a) / N
  return integral

In [ ]:
N = [1000, 3000, 5000, 10000, 15000]
a = 1
b = 3
for n in N:
  start = time.time()
  integral = task2(a, b, n)
  print("N = ", n, ". Elapsed time (with compilation): ", (time.time() - start), "s.", sep='')

N = 1000. Elapsed time (with compilation): 0.19311261177062988s.
N = 3000. Elapsed time (with compilation): 1.0967254638671875e-05s.
N = 5000. Elapsed time (with compilation): 1.0728836059570312e-05s.
N = 10000. Elapsed time (with compilation): 1.8596649169921875e-05s.
N = 15000. Elapsed time (with compilation): 2.7179718017578125e-05s.


In [ ]:
def task2_cpu(a, b, N):
  integral = (a**3 + b**3 - 6) / 2
  for i in range(1, N-1):
    integral += ((a + i * (b - a))**3 - 3) / N
  integral *= (b - a) / N
  return integral

In [ ]:
for n in N:
  start = time.time()
  integral = task2_cpu(a, b, n)
  print("N = ", n, ". Elapsed time (with compilation): ", (time.time() - start), "s.", sep='')

N = 1000. Elapsed time (with compilation): 0.0006968975067138672s.
N = 3000. Elapsed time (with compilation): 0.0033292770385742188s.
N = 5000. Elapsed time (with compilation): 0.0030450820922851562s.
N = 10000. Elapsed time (with compilation): 0.009258270263671875s.
N = 15000. Elapsed time (with compilation): 0.013104438781738281s.
